In [1]:
from keras.preprocessing.image import ImageDataGenerator

from keras.applications import VGG16
from keras.layers import GlobalMaxPooling2D, Dense, Dropout
from keras.models import Model
from tensorflow.keras.optimizers.legacy import Adam

In [2]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=30
)

training_set = train_datagen.flow_from_directory(
    '../dataset/training_set',
    target_size=(64, 64),
    batch_size=32,
    shuffle=True,
    class_mode='binary')

Found 3110 images belonging to 2 classes.


In [3]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory(
    '../dataset/test_set',
    target_size=(64,64),
    batch_size=32,
    shuffle=True,
    class_mode='binary',)

Found 240 images belonging to 2 classes.


In [4]:
def create_VGG16(input_shape=(64, 64, 3), lr=0.001, dropout=0.4, units=512):
    pre_trained_model = VGG16(input_shape=input_shape, include_top=False, weights="imagenet")
    
    for layer in pre_trained_model.layers[:15]:
        layer.trainable = False

    for layer in pre_trained_model.layers[15:]:
        layer.trainable = True
    
    last_layer = pre_trained_model.get_layer('block5_pool')
    last_output = last_layer.output
    
    x = GlobalMaxPooling2D()(last_output)
    x = Dense(units, activation='relu')(x)
    x = Dropout(dropout)(x)
    x = Dense(1, activation='sigmoid')(x)

    model = Model(pre_trained_model.input, x)

    adam = Adam(learning_rate=lr)

    model.compile(loss='binary_crossentropy',
                  optimizer=adam,
                  metrics=['accuracy'])
    
    return model


In [5]:
best_accuracy = 0
best_hyperparameters = {}

learning_rates = [0.001]
dropout = [0.4]
epochs = [1]
units = [512, 1024]

i = 0

for lr in learning_rates:
    for dout in dropout:
        for epoch in epochs:
            for unit in units:
                vgg16 = create_VGG16(input_shape=(64, 64, 3), lr=lr, dropout=dout, units=unit)
                vgg16.fit(x=training_set, validation_data=test_set, epochs=epoch, verbose=0)
                accuracy = vgg16.evaluate(test_set, verbose=0)[1]
                loss = vgg16.evaluate(test_set, verbose=0)[0]
                model_json = vgg16.to_json()
                with open(str(i) + str(accuracy) + ".json", "w") as json_file:
                    json_file.write(model_json)
                vgg16.save_weights(str(i) + str(accuracy) + ".h5")
                print("Saved model to disk")    
                i=i+1
                print(f'Hiperparameters: learning_rate: {lr}, dropout_value: {dout}, epochs: {epoch}, units: {unit}. Accuracy: {accuracy}, Loss: {loss}')
                if(accuracy > best_accuracy):
                    best_accuracy = accuracy
                    best_hyperparameters = {lr, dout, epoch, unit}

print(f'Best Hyperparameters: {best_hyperparameters} with accuracy: {best_accuracy}')

Saved model to disk
Hiperparameters: learning_rate: 0.001, dropout_value: 0.4, epochs: 1, units: 512. Accuracy: 0.762499988079071, Loss: 0.4585930109024048


KeyboardInterrupt: 